In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

# TODO
### PUT FUNCTIONS TO PYTHON SCRIPT, OUTPUT SOME EXAMPLE IN NOTEBOOK AND ADD DOCSTRING

In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

In [2]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [3]:
market_data

Time (UTC+10)  Regions NSW Trading Price ($/MWh)  \
0     2018-01-01 00:30:00                              91.86   
1     2018-01-01 01:00:00                              88.83   
2     2018-01-01 01:30:00                              73.62   
3     2018-01-01 02:00:00                              71.49   
4     2018-01-01 02:30:00                              69.27   
...                   ...                                ...   
63451 2021-08-14 22:00:00                              50.84   
63452 2021-08-14 22:30:00                              66.85   
63453 2021-08-14 23:00:00                              55.64   
63454 2021-08-14 23:30:00                              52.25   
63455 2021-08-15 00:00:00                              48.69   

       Regions SA Trading Price ($/MWh)  Regions TAS Trading Price ($/MWh)  \
0                                107.17                              92.28   
1                                103.31                              87.53   
2                                 88.20                              76.29   
3                                 85.24                              75.10   
4                                 81.75                              72.92   
...                                 ...                                ...   
63451                             59.16                               7.54   
63452                             80.01                              10.52   
63453                             77.76                               7.63   
63454                             76.47                               7.52   
63455                             83.68                              23.87   

       Regions VIC Trading Price ($/MWh)  \
0                                  92.46   
1                                  87.62   
2                                  73.08   
3                                  70.18   
4                                  67.43   
...                                  ...   
63451                              49.93   
63452                              62.86   
63453                              32.26   
63454                              25.10   
63455                               8.73   

       Regions NSW Trading Total Intermittent Generation (MW)  \
0                                                   0.15        
1                                                   0.13        
2                                                   0.14        
3                                                   0.16        
4                                                   0.16        
...                                                  ...        
63451                                               8.74        
63452                                               7.52        
63453                                               7.69        
63454                                               8.35        
63455                                               8.07        

       Regions SA Trading Total Intermittent Generation (MW)  \
0                                                  43.07       
1                                                  41.67       
2                                                  42.15       
3                                                  38.31       
4                                                  33.39       
...                                                  ...       
63451                                              36.42       
63452                                              51.83       
63453                                              42.04       
63454                                              38.04       
63455                                              47.47       

       Regions TAS Trading Total Intermittent Generation (MW)  \
0                                                 118.73        
1                                                 110.48        
2                               

### Add Period as a column

In [4]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['period'] = pd.Series(period)
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])

### Battery Class

In [5]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# NOTE : IF THERE IS ANY CHANGE IN CALCULATION, CHANGE CALCULATION MARKED BY (BATTERY CALCULATION).
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs.
#      - Setting : Set all the battery functionalities. (BATTERY CALCULATION)
#      - FirstOptimisation : Ensure that energy are not wasted or not used. (BATTERY CALCULATION)
#      - SecondOptimisation : Ensure that to always charge the highest price less and discharge
#                             highest price more. (BATTERY CALCULATION)
#
# Created by: Gilbert
###################################
class Battery:
    # Battery Specifications
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
        
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price): 
        # Charge and Discharge Period
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        # Spot Price during Charge and Discharge
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
    
    ########################################################################################    
                
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        return discharge_revenue + charge_revenue
    
    ########################################################################################
    
    def Setting(self):
        
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity = [[0, 0] for i in range(len_charge)]
        self.discharge_capacity = [[0, 0] for i in range(len_discharge)]
        
        # CHARGE PERIOD --------------------------------------------------------------------
        for t in range(len_charge):
            # RAW_POWER[t] = -MIN(BATTERY_POWER, (BATTERY_CAPACITY - OPENING_CAPACITY[t]) / CHARGE_EFFICIENCY * 2)
            self.charge_raw_power[t] = -min(battery_power, 
                                           (battery_cap - self.charge_capacity[t][OPENING]) / 
                                            self.charge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER / 2
            self.charge_market_dispatch[t] = self.charge_raw_power[t] / 2

            # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
            #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                        self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                        battery_cap))

            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_charge:
                self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------

        # Set DISCHARGE CAPACITY AT t = 0 to be the the last t of CHARGING CAPACITY
        self.discharge_capacity[0][OPENING] = self.charge_capacity[-1][CLOSING]

        for t in range(len_discharge):
            # RAW_POWER[t] = MIN(BATTERY_POWER, OPENING_CAPACITY[t] * 2)
            self.discharge_raw_power[t] = min(battery_power, self.discharge_capacity[t][OPENING] * 2)

            # MARKET_DISPATCH[t] = RAW_POWER[t] / 2 * DISCHARGE EFFICIENCY
            self.discharge_market_dispatch[t] = self.discharge_raw_power[t] / 2 * self.discharge_efficiency

            # CLOSING CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] -
            #                        MARKET_DISPATCH[t] * (1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] -
                                                            self.discharge_market_dispatch[t] * (1 / self.discharge_efficiency),
                                                            battery_cap))
            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_discharge:
                self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
    
    ########################################################################################
        
    def FirstOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        if (len_charge >= MAX_CHARGE_PERIOD and len_discharge >= MAX_DISCHARGE_PERIOD) or (len_charge == len_discharge):
            #md = 117
            #self.discharge_raw_power[-1] = min(md * 2 / self.discharge_efficiency, self.discharge_raw_power[-1])
            #self.discharge_market_dispatch[-1] = self.discharge_raw_power[-1] / 2 * self.discharge_efficiency
            return
        elif (len_charge - len_discharge == 1):
            self.discharge_capacity[-1][CLOSING] = 0
            # OPENING_CAPACITY[-1] = MARKET_DISPATCH / DISCHARGE_EFFICIENCY
            self.discharge_capacity[-1][OPENING] = self.discharge_market_dispatch[-1] / self.discharge_efficiency
            for t in range(1, len_discharge):
                # CLOSING_CAPACITY[-t - 1] = OPENING_CAPACITY[-t]
                self.discharge_capacity[-t - 1][CLOSING] = self.discharge_capacity[-t][OPENING]
                # OPENING_CAPACITY[-t - 1] = CLOSING_CAPACITY[-t - 1] + MARKET_DISPATCH[-t - 1] / DISCHARGE_EFFICIENCY
                self.discharge_capacity[-t - 1][OPENING] = self.discharge_capacity[-t - 1][CLOSING] + self.discharge_market_dispatch[-t - 1] / self.discharge_efficiency
            
            # CLOSING_CAPACITY[-1] = OPENING_CAPACITY[0]
            self.charge_capacity[-1][CLOSING] = self.discharge_capacity[0][OPENING]
            # MARKET_DISPATCH[-1] = -(CLOSING_CAPACITY[-1] - OPENING_CAPACITY[-1]) / CHARGE_EFFICIENCY
            self.charge_market_dispatch[-1] = -(self.charge_capacity[-1][CLOSING] - self.charge_capacity[-1][OPENING]) / self.charge_efficiency
            # RAW_POWER[-1] = MARKET_DISPATCH[-1] * 2
            self.charge_raw_power[-1] = self.charge_market_dispatch[-1] * 2
            
    ########################################################################################

    def SecondOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        # CHARGE PERIOD --------------------------------------------------------------------
        highest_price_index = np.array(self.charge_period).argmax(axis=0)[0]
        lowest_charge_index = self.charge_market_dispatch.index(max(self.charge_market_dispatch))    
        
        # IF THE HIGHEST CHARGING PRICE DOESN'T HAVE THE LOWEST CHARGING RATE, SWAP!
        if (highest_price_index != lowest_charge_index):
            tmp = self.charge_market_dispatch[highest_price_index]
            self.charge_market_dispatch[highest_price_index] = self.charge_market_dispatch[lowest_charge_index]
            self.charge_market_dispatch[lowest_charge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_charge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2
                self.charge_raw_power[t] = self.charge_market_dispatch[t] * 2

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                            self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_charge:
                    self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------
        lowest_price_index = np.array(self.discharge_period).argmax(axis=0)[0] 
        min_discharge_index = self.discharge_market_dispatch.index(min(self.discharge_market_dispatch))   
        
        # IF THE LOWEST DISCHARGING PRICE DOESN'T HAVE THE LOWEST DISCHARGING RATE, SWAP!
        if (lowest_price_index != min_discharge_index):
            tmp = self.discharge_market_dispatch[lowest_price_index]
            self.discharge_market_dispatch[lowest_price_index] = self.discharge_market_dispatch[min_discharge_index]
            self.discharge_market_dispatch[min_discharge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_discharge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2 / DISCHARGE_EFFICIENCY
                self.discharge_raw_power[t] = self.discharge_market_dispatch[t] * 2 / self.discharge_efficiency

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - 
                                                            self.discharge_market_dispatch[t] / self.discharge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_discharge:
                    self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
        

### Helper function for battery optimisation

In [60]:
###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods = False, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    if (selected_periods):
        retrieved_prices = []
        for period in selected_periods:
            # append(minimum or maximum ranking, spot_price[index])
            retrieved_prices.append((period[1], spot_price[period[1] - 1]))
        return retrieved_prices
    else:
        return spot_price

########################################################################################

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Efficiency: O(3N + NLogN) = O(NLogN)
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 5, sell_threshold = 4):
    EMPTY = ' '
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:len(price)] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

########################################################################################

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    EMPTY = ' '
    
    period = len(buy_period)
    
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != EMPTY:
            # If battery buying point period is less MAXIMUM SELLING 
            # PERIOD PER PAIR, add new period.
            if len(sell) < MAX_SELL_PERIOD:
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            # else, if battery selling point is full and there is 
            # higher maximum selling point then remove the lowest
            # selling point and add the new one into Dictionary.    
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and buy_period[p] != EMPTY:
            # If battery buying point period is less MAXIMUM BUYING 
            # PERIOD PER PAIR, add new period.
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != EMPTY and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        
    return battery

########################################################################################

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs, region = "VIC"):    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)

            battery = Battery(buy_period, buy_price, sell_period, sell_price)
            battery.Setting()
            battery.FirstOptimisation()
            battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

########################################################################################

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - Daily revenues
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

########################################################################################

###################################
# A function to optimise battery charging and discharging period. This is where 
# mainly the optimisations are performed with the helper functions.
#
# Parameters:
#      - daily_data : the targeted daily dataset, minimum dataset length of 48.
#
# Return:
#      - raw_power : List of Daily Raw Power for charging and discharging
#      - market_dispatch : List of Daily Market Dispatch for charging and discharging
#      - opening_capacity : List of Daily Opening Capacity for charging and discharging
#      - closing_capacity : List of Daily Closing Capacity for charging and discharging
#
# Efficiency: O(N^2 * NLogN) = O(N^3LogN)
# Created by: Gilbert
###################################
def PeriodOptimisation(given_data, region = "VIC"):
    best_batteries = OrderedDict() # Initialise an Ordered Dictionary
    
    # TODO: OPTIMISE EFFICIENCY HERE! Make this at least < N^2
    # Iterate over all possible combinations of battery pairs. (O(N^2))
    period = len(given_data)
    count = 0
    for s in range(1, period + 1):
        for b in range(1 , period + 1): # change this to range(1, period - s + 1) to reduce by half
            # Get the minimum and maximum price based on the given threshold
            min_price, max_price = GetMinMax(given_data, buy_threshold = b, sell_threshold = s, region = region)
            # Get the battery pairs based on minimum and maximum price
            battery_pairs = FindBatteryPairs(min_price, max_price)
            # Get battery optimisation for the selected threshold
            all_batteries = SetChargeDischarge(given_data, battery_pairs, region = region)
            # Compute daily revenues of selected battery combinations
            dailyrev = ComputeDailyRevenue(all_batteries)
            #print((b, s), dailyrev)
            if dailyrev < 0:
                break
            count += 1
            # Insert revenue as key, batteries combination and threshold as value
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))
            #print(count)
    #print(count)           
    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    best_threshold = best_batteries[best_revenue][1]
    #print(best_threshold)
    battery = best_batteries[best_revenue][0] # The Best battery combinations
    
    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(period)]
    market_dispatch = [0 for i in range(period)]
    opening_capacity = [0 for i in range(period)]
    closing_capacity = [0 for i in range(period)]
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
    
    # Formatting the opening and closing capacity.
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != 0.0 and opening_capacity[i] == 0.0:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]
                
    return raw_power, market_dispatch, opening_capacity, closing_capacity, best_revenue

def FirstOptimisation(data, period = 48, region = 'VIC'):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []

    start = 0
    end = period 

    while end <= len(data):
        tmp_data = data.iloc[start:end, :]
        daily_rp, daily_md, daily_oc, daily_cc, _ = PeriodOptimisation(tmp_data)
        raw_power.extend(daily_rp)
        market_dispatch.extend(daily_md)
        opening_capacity.extend(daily_oc)
        closing_capacity.extend(daily_cc)

        start += period
        end += period

    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

### Check Dependency

In [39]:
data = pd.read_excel("../../preprocessed_data/First Algorithm/SM.xlsx")
data = data[['Time', 'Price', 'Status', 'Actual', 'Opening Capacity', 'Closing Capacity']]

In [40]:
data.drop(index=market_data.index[0], axis=0, inplace=True)
data = data.reset_index(drop = True)
data

Time  Price      Status  Actual  Opening Capacity  \
0     2018-01-01 00:30:00  92.46  Do Nothing     0.0                 0   
1     2018-01-01 01:00:00  87.62  Do Nothing     0.0                 0   
2     2018-01-01 01:30:00  73.08  Do Nothing     0.0                 0   
3     2018-01-01 02:00:00  70.18  Do Nothing     0.0                 0   
4     2018-01-01 02:30:00  67.43  Do Nothing     0.0                 0   
...                   ...    ...         ...     ...               ...   
63451 2021-08-14 22:00:00  49.93  Do Nothing     0.0               580   
63452 2021-08-14 22:30:00  62.86  Do Nothing     0.0               580   
63453 2021-08-14 23:00:00  32.26  Do Nothing     0.0               580   
63454 2021-08-14 23:30:00  25.10  Do Nothing     0.0               580   
63455 2021-08-15 00:00:00   8.73  Do Nothing     0.0               580   

       Closing Capacity  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  
...                 ...  
63451               580  
63452               580  
63453               580  
63454               580  
63455               580  

[63456 rows x 6 columns]

In [41]:
PERIOD = 48

def getDependency(dataframe):
    dayBorder = []
    
    dependency = []
    for i in range(PERIOD, len(dataframe), PERIOD):
        dayBorder.append((i - 1, i))
        if dataframe.loc[i - 1, 'Closing Capacity'] != 0 and dataframe.loc[i, 'Closing Capacity'] != 0:
            dependency.append((i - 1, i))
            
    consecutiveDependency = []
    tmp = []
    for d in range(len(dependency) - 1):
        curr_d = dependency[d][0]
        next_d = dependency[d + 1][0]
        tmp.append(dependency[d])
        if (next_d - curr_d != PERIOD):
            consecutiveDependency.append(tmp)
            tmp = []
       
    if dependency[-1][0] - consecutiveDependency[-1][0][0] == PERIOD:
        consecutiveDependency[-1].append(dependency[-1])
    else:
        consecutiveDependency.append([dependency[-1]])

    for c in range(len(consecutiveDependency)):
        current = consecutiveDependency[c]
        if len(current) != 1:
            consecutiveDependency[c] = [current[0], current[-1]]
            
    return consecutiveDependency

def createPeriod(current, until):
    tmp = []
    for i in range(current, until, PERIOD):
        tmp.append((i, i + PERIOD - 1))
    return tmp

def getTimePeriod(dataframe):
    dependency = getDependency(dataframe)
    
    tmp = []
    for depend in dependency:
        openPeriod = depend[0][0] - PERIOD + 1     # 48th period - 47 = 1st period
        closePeriod = depend[-1][-1] + PERIOD - 1  # 1st period + 47 = 48th period
        tmp.append((openPeriod, closePeriod))
        
    timeIndex = []
    timeIndex.extend(createPeriod(0, tmp[0][0]))
    
    for t in range(1, len(tmp)):
        timeIndex.append((tmp[t-1]))
        timeIndex.extend(createPeriod(tmp[t-1][1] + 1, tmp[t][0]))
    timeIndex.append(tmp[t])  
    
    timePeriod = []
    for index in timeIndex:
        timePeriod.append((dataframe.iloc[index[0], 0], dataframe.iloc[index[1], 0], index[1] - index[0] + 1))
   
    return timePeriod

In [49]:
timePeriod = getTimePeriod(data)
timePeriod[5]

(Timestamp('2018-01-07 00:30:00'), Timestamp('2018-01-09 00:00:00'), 96)

In [58]:
def SecondOptimisation(data, timePeriod):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []
    
    count = 0
    for time in timePeriod:
        start_t = time[0]
        end_t = time[1]
        period_t = time[2]
        
        data_interval = data.loc[(data['Time (UTC+10)'] >= start_t) & \
                                 (data['Time (UTC+10)'] <= end_t)]
        
        tmp_rev = 0
        if period_t > PERIOD:
            start = 0
            end = PERIOD 
            tmp_rp = []
            tmp_md = []
            tmp_oc = []
            tmp_cc = []
            while end <= len(data_interval):
                tmp_data = data_interval.iloc[start:end, :]
                daily_rp, daily_md, daily_oc, daily_cc, revenue_1 = PeriodOptimisation(tmp_data)
                tmp_rev += revenue_1
                tmp_rp.extend(daily_rp)
                tmp_md.extend(daily_md)
                tmp_oc.extend(daily_oc)
                tmp_cc.extend(daily_cc)
                
                start += PERIOD
                end += PERIOD
            
        daily_rp, daily_md, daily_oc, daily_cc, revenue_2 = PeriodOptimisation(data_interval)

        if tmp_rev != 0 and tmp_rev > revenue_2:
            print(tmp_rev, revenue_2)
            raw_power.extend(tmp_rp)
            market_dispatch.extend(tmp_md)
            opening_capacity.extend(tmp_oc)
            closing_capacity.extend(tmp_cc)
        else:
            raw_power.extend(daily_rp)
            market_dispatch.extend(daily_md)
            opening_capacity.extend(daily_oc)
            closing_capacity.extend(daily_cc)
            
        print(count)
        count += 1
    
    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

In [61]:
vic_spot_price = market_data[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]].copy()
example = SecondOptimisation(vic_spot_price, timePeriod)
example.to_excel('../../preprocessed_data/First Algorithm/example_1.xlsx', index = False)

0
1
2
3
4
5
6
7
8
9
40315.0621155124 39949.57708595472
10
11
12
2456604.9476628657 2449142.4274247987
13
477336.38823714596 473748.1805942415
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
65035.99850723457 61533.780995434485
30
31
32
33
34
35
33999.95693877454 33829.77307735174
36
37
38
39
40
41
42
43
44
45
131992.51011226652 123617.01266961655
46
47
147215.8011996911 136280.94880645137
48
49
50
64974.04203282094 61388.739420534795
51
52
110099.41403213027 99150.0168317031
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
88522.08073333165 87529.55657507737
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
407762.77902710496 392354.94104182976
85
86
87
88
118558.88707248404 117694.03289035207
89
265048.29347490805 248057.73157628148
90
91
92
93
94
95
96
97
98
99
100
101
102
103
261002.07464943995 260986.82901720094
104
297867.04421969 278802.11901774973
105
106
107
108
109
110
111
112
113
114
115
375990.3600879521 369141.6616808465
116
117
118
186858.6868651732 185275.154936707
119
120
1

89422.82931350768 87704.90365233042
860
106177.27695009139 102924.57296448591
861
862
153883.85910273797 152070.6946368214
863
864
865
866
867
868
869
84423.07990739265 84240.53036478866
870
871
872
873
874
127000.2256722794 125173.41037490075
875
876
877
878
879
880
881
321508.2975770042 318893.6647314189
882
395130.1886633519 318354.5475976354
883
884
885
886
349922.7208130743 349395.3716164211
887
888
131303.39885138636 120217.47445185672
889
890
891
892
893
894
895
896
897
649107.2777500751 648252.1355812188
898
899
900
3576230.7915781597 3562454.914363035
901
902
903
904
905
906
907
908
257514.01640418824 235920.96334134098
909
910
911
1154124.9528179236 1122189.0767841754
912
913
914
915
916
917
918
919
730463.7386930827 691162.3959299182
920
468869.24859132525 439192.84967763367
921
922
919786.2679360365 849967.6110806661
923
924
925
926
217238.20285729342 194855.22941066377
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
339413.06456709834 257191.339

In [54]:

data.to_excel('../../preprocessed_data/First Algorithm/2ndOpti.xlsx', index = False)

KeyError: "None of [Index(['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 'Raw Power (MW)',\n       'Market Dispatch (MWh)', 'Opening Capacity (MWh)',\n       'Closing Capacity (MWh)'],\n      dtype='object')] are in the [columns]"

In [182]:
test_start_period = timePeriod[2][0]
test_end_period = timePeriod[2][1]

vic_spot_price = market_data[['Time (UTC+10)', 'period', 'Regions VIC Trading Price ($/MWh)']]

test = vic_spot_price.loc[(vic_spot_price['Time (UTC+10)'] >= test_start_period) & \
        (vic_spot_price['Time (UTC+10)'] <= test_end_period)].copy()

test = test.reset_index(drop = True)

test = FirstOptimisation(test, period = len(test))
test.to_csv('../../preprocessed_data/First Algorithm/test.csv', index = False)

test

Time (UTC+10)  period  Regions VIC Trading Price ($/MWh)  \
0  2018-01-03 00:30:00       1                              59.33   
1  2018-01-03 01:00:00       2                              55.01   
2  2018-01-03 01:30:00       3                              46.83   
3  2018-01-03 02:00:00       4                              46.92   
4  2018-01-03 02:30:00       5                              49.74   
5  2018-01-03 03:00:00       6                              49.37   
6  2018-01-03 03:30:00       7                              50.99   
7  2018-01-03 04:00:00       8                              52.00   
8  2018-01-03 04:30:00       9                              54.33   
9  2018-01-03 05:00:00      10                              55.38   
10 2018-01-03 05:30:00      11                              62.36   
11 2018-01-03 06:00:00      12                              63.41   
12 2018-01-03 06:30:00      13                              67.53   
13 2018-01-03 07:00:00      14                              69.86   
14 2018-01-03 07:30:00      15                              71.44   
15 2018-01-03 08:00:00      16                              71.00   
16 2018-01-03 08:30:00      17                              72.51   
17 2018-01-03 09:00:00      18                              73.81   
18 2018-01-03 09:30:00      19                              74.10   
19 2018-01-03 10:00:00      20                              73.46   
20 2018-01-03 10:30:00      21                              71.70   
21 2018-01-03 11:00:00      22                              75.72   
22 2018-01-03 11:30:00      23                              73.89   
23 2018-01-03 12:00:00      24                              76.72   
24 2018-01-03 12:30:00      25                              73.10   
25 2018-01-03 13:00:00      26                              72.73   
26 2018-01-03 13:30:00      27                              72.04   
27 2018-01-03 14:00:00      28                              76.79   
28 2018-01-03 14:30:00      29                              73.56   
29 2018-01-03 15:00:00      30                              74.14   
30 2018-01-03 15:30:00      31                              70.81   
31 2018-01-03 16:00:00      32                              72.27   
32 2018-01-03 16:30:00      33                              70.84   
33 2018-01-03 17:00:00      34                              71.56   
34 2018-01-03 17:30:00      35                              69.83   
35 2018-01-03 18:00:00      36                              66.88   
36 2018-01-03 18:30:00      37                              73.66   
37 2018-01-03 19:00:00      38                              76.22   
38 2018-01-03 19:30:00      39                              77.83   
39 2018-01-03 20:00:00      40                              71.99   
40 2018-01-03 20:30:00      41                              66.75   
41 2018-01-03 21:00:00      42                              63.98   
42 2018-01-03 21:30:00      43                              63.74   
43 2018-01-03 22:00:00      44                              62.66   
44 2018-01-03 22:30:00      45                              62.53   
45 2018-01-03 23:00:00      46                              63.84   
46 2018-01-03 23:30:00      47                              68.42   
47 2018-01-04 00:00:00      48                              69.84   

    Raw Power (MW)  Market Dispatch (MWh)  Opening Capacity (MWh)  \
0         0.000000               0.000000                     0.0   
1         0.000000               0.000000                     0.0   
2      -300.000000            -150.000000                     0.0   
3      -300.000000            -150.000000                   135.0   
4      -300.000000            -150.000000                   270.0   
5      -300.000000            -150.000000                   405.0   
6       -88.888889             -44.444444                   540.0   
7         0.000000               0.000000                   580.0   
8     

---

In [ ]:
market_data = FirstOptimisation(market_data, period = len(market_data))

In [ ]:
vic_data = market_data[['Time (UTC+10)', 'period', 'Regions VIC Trading Price ($/MWh)', 'Raw Power (MW)', 
                        'Market Dispatch (MWh)', 'Opening Capacity (MWh)', 'Closing Capacity (MWh)']]
vic_data.to_excel('../../preprocessed_data/First Algorithm/Victoria_data_whole.xlsx', index = False)